In [1]:
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain_community.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.tools import BaseTool
from subprocess import Popen,PIPE
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
openai = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [3]:
if 'first-index' not in pc.list_indexes().names():
    pc.create_index(
        name='first-index', 
        dimension=1536, 
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-west-2'
        )
    )

In [4]:
""" 
loader = PyPDFLoader("cybersecuirty_sb_factsheets_all.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
docs = text_splitter.split_documents(pages)

"""

' \nloader = PyPDFLoader("cybersecuirty_sb_factsheets_all.pdf")\npages = loader.load_and_split()\n\ntext_splitter = RecursiveCharacterTextSplitter(\n    chunk_size = 1000,\n    chunk_overlap  = 200,\n    length_function = len,\n)\ndocs = text_splitter.split_documents(pages)\n\n'

In [5]:

model_name = 'text-embedding-ada-002'
embeddings = OpenAIEmbeddings(model = model_name, openai_api_key=os.getenv('OPENAI_API_KEY'))
index_name = 'first-index'


In [6]:
""" 
from langchain.vectorstores import Pinecone
vector_store = Pinecone.from_texts([t.page_content for t in docs],embeddings, index_name=index_name)
"""

' \nfrom langchain.vectorstores import Pinecone\nvector_store = Pinecone.from_texts([t.page_content for t in docs],embeddings, index_name=index_name)\n'

In [7]:
from langchain.vectorstores import Pinecone
vector_store = Pinecone.from_existing_index(index_name,embeddings)


In [8]:
query = "How to protect your business?"
docs = vector_store.similarity_search(query)


In [9]:
for i in range(len(docs)):
    print(docs[i].page_content, "*"*100)


CYBERSECURITY FOR
SMALL BUSINESSHOW TO  PROTECT  YOUR BUSINESS
Have a plan 
How would your business stay up and running after a ransomware attack? 
Put this plan in writing and share it with everyone who needs to know.
Back up your data 
Regularly save important files to a drive or server that’s not connected to your network. Make data backup part of your routine business operations.
Keep your security up to date 
Always install the latest patches and updates. Look for additional means of protection, like email authentication, and intrusion prevention software, and set them to update 
automatically on your computer. On mobile devices, you may have to do it manually.
Alert your staff 
Teach them how to avoid phishing scams and show them some of the common ways 
computers and devices become infected. Include tips for spotting and protecting against 
ransomware in your regular orientation and training.
WHAT TO 
DO IF YOU’RE
ATTACKEDLimit the damage
Immediately disconnect *****************

In [10]:
print(docs[0].page_content, "*"*100)

CYBERSECURITY FOR
SMALL BUSINESSHOW TO  PROTECT  YOUR BUSINESS
Have a plan 
How would your business stay up and running after a ransomware attack? 
Put this plan in writing and share it with everyone who needs to know.
Back up your data 
Regularly save important files to a drive or server that’s not connected to your network. Make data backup part of your routine business operations.
Keep your security up to date 
Always install the latest patches and updates. Look for additional means of protection, like email authentication, and intrusion prevention software, and set them to update 
automatically on your computer. On mobile devices, you may have to do it manually.
Alert your staff 
Teach them how to avoid phishing scams and show them some of the common ways 
computers and devices become infected. Include tips for spotting and protecting against 
ransomware in your regular orientation and training.
WHAT TO 
DO IF YOU’RE
ATTACKEDLimit the damage
Immediately disconnect *****************

In [11]:
# chat completion llm
llm = ChatOpenAI(
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    model_name='gpt-3.5-turbo-1106',
    temperature=0.0
)
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [12]:
result = qa.invoke(query)["result"]

In [13]:
for res in result.split("\n"):
    if res :
        print(res)

To protect your business, you should have a plan in place for staying up and running after a ransomware attack, regularly back up your data, keep your security up to date by installing the latest patches and updates, alert your staff about phishing scams, and deploy a safety net using email authentication technology to prevent phishing emails from reaching your network. Additionally, you should update your software, secure your files, require passwords for devices, encrypt devices and media containing sensitive information, use multi-factor authentication, and create a culture of security through regular employee training. It's also important to have a plan for saving data, running the business, and notifying customers in case of a breach. For more information, you can visit FTC.gov/SmallBusiness.


In [14]:
class ExecuteCommand(BaseTool):
    name : str =  "execute command",
    description = "execute commands in the VM"

    def _run(self, command: str):
        cmd = command.split()
        if cmd[0]=='sudo':
            cmd.insert(1,'-S')
        #print(cmd)        
        p = Popen(cmd ,stdin = PIPE, stdout = PIPE, universal_newlines = True)
        p.stdin.write('test1!')
        res,_ = p.communicate()
    
        p.terminate()

        return res
        
        #return json.dumps(cmd)       

    def _arun(self, webpage: str):
        raise NotImplementedError("This tool does not support async")

execute_command = ExecuteCommand()

In [15]:
class Transfrom_to_Bash(BaseTool):
    name : str = "Transform to Bash",
    description= "Give bash command to be executed in the VM"

    def _run(self, command: str):
        return command

    def _arun(self, webpage: str):
        raise NotImplementedError("This tool does not support async")

TransfromtoBash = Transfrom_to_Bash()

In [16]:
from langchain.agents import Tool

tools = [
    Tool(
        name = 'Knowledge Base',
        func=qa.invoke,
        description=(
            """use this tool when answering general knowledge queries to get more information about the topic."""
        )
    ),
    Tool(
        name = 'Execute Command',
        func=execute_command._run,
        description=(
            """use this tool to execute commands in the VM"""
        )
    ),
    Tool(
        name = 'Transform to Bash',
        func=TransfromtoBash._run,
        description=(
            """use this tool to transform text to bash command"""
        )
    )
    
]

In [17]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,
    early_stopping_method='generate',
    memory=conversational_memory
)

c:\Users\zikri\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [18]:
fixed_prompt = """ 
            Start looking for the answer by using the tools available first. You have to answer the question with a detailed plan by putting on points. 
            If you can, instead of text, return bash commands to be executed on a VM. Remember answer question by returning bash commands according to the question.
            """
agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [19]:
inp = []
memory =[]
out = []
while True:
    query = input("Ask me anything: ")
    if query =='':
        break
    rest = agent(query)
    inp.append(rest['input'])
    memory.append(rest['chat_history'])
    out.append(rest['output'])    
    

c:\Users\zikri\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "How to protect your business"
}
```
Observation: {'query': 'How to protect your business', 'result': "To protect your business, you should have a plan in place for staying up and running after a ransomware attack, regularly back up your data offline, keep your security up to date by installing the latest patches and updates, and alert your staff about phishing scams and how to avoid them. Additionally, you can deploy a safety net by using email authentication technology to help prevent phishing emails from reaching your employees. It's also important to update your software, secure your files, require passwords for devices, encrypt devices, and implement multi-factor authentication. Lastly, create a culture of security by implementing regular employee training and have a plan for saving data, running the business, and notifying customers in case of a breach. For more information, you 

In [20]:
import regex as re
sents = re.sub(r'\n[0-9]+\.\s', r'\n\n\n\n', rest['output'])

#[sent.strip() for sent in re.split(r'\s[0-9]+\.',rest['output'])]

[sent.strip() for sent in re.split(r'\n\n\n\n',sents)]

["I'm sorry, I cannot execute commands directly in the VM. However, I can provide you with the necessary bash commands for specific tasks. Please let me know what specific task or command you need assistance with."]

In [21]:
type(rest)

dict

In [22]:
rest.keys()

dict_keys(['input', 'chat_history', 'output'])

In [23]:
rest['output']

"I'm sorry, I cannot execute commands directly in the VM. However, I can provide you with the necessary bash commands for specific tasks. Please let me know what specific task or command you need assistance with."

In [24]:
rest['chat_history']

[HumanMessage(content='How to protect your business?'),
 AIMessage(content="To protect your business, you should have a plan in place for staying up and running after a ransomware attack, regularly back up your data offline, keep your security up to date by installing the latest patches and updates, and alert your staff about phishing scams and how to avoid them. Additionally, you can deploy a safety net by using email authentication technology to help prevent phishing emails from reaching your employees. It's also important to update your software, secure your files, require passwords for devices, encrypt devices, and implement multi-factor authentication. Lastly, create a culture of security by implementing regular employee training and have a plan for saving data, running the business, and notifying customers in case of a breach. For more information, you can visit FTC.gov/SmallBusiness."),
 HumanMessage(content='bash command'),
 AIMessage(content='To protect your business, you shou